# Introducción

- El "dataframe" usado se encuentra en https://www.kaggle.com/competitions/diamonds-part-datamad0122/

- La fuente original es: Tiffany & Co's snapshot pricelist from 2017

- Es parte de una competición cerrada sobre diamantes con sus características. La "target" es el precio de los diamantes

- Se tiene como objetivo de este proyecto superar la cifra de la predicción ganadora


# EDA

- Es el análisis exploratorio de los datos

- Se lleva a cabo en el "notebook" homónimo


## Observaciones iniciales

- Se adaptan los nombres de las columnas para que sean legibles

- No se detectan nulos

- Se encuentran duplicados, pero como todos tienen su propia identificación se asume que cada diamante es único y deben conservarse

- Se nota que hay diamantes con 0 milimetros de altura, anchura y/o profundidad

## Cambios probados

- Se detectan una serie de características:

1) Varias columnas tienen unos pocos valores atípicos extremadamente altos ["depth (percentage)", "table (percentage)", "width (millimeters)", "depth (millimeters)"]

2) Como se ha anticipado, varias filas tienen un cero en todas las variables relacionadas con el tamaño de los diamantes, sin contar el peso ["lenght (millimeters)", "width (millimeters)" y "depth (millimeters)"].

3) Dos columnas con una alta correlación negativa tienen unos pocos "outliers" que destacan moderadamente y que, además, son compartidos ["depth (percentage)" y "table (percentage)"].

4) Si se borrasen las filas con ceros del punto 2, todavía quedaría un cero en una fila para el "lenght". Como los diamantes son prácticamente circulares, "lenght" y "width" son casi idénticas ["lenght (millimeters)"].

5) Si se borrasen las filas con ceros del punto 2 y se solucionase el cero del punto 4, todavía quedaría un cero en una fila para "depth (millimeters)". Se descubre que la variable "depth (percentage)" se obtiene de dividir ese "depth (millimeters)" entre la media de "width" y "lenght" ["depth (millimeters)"].

6) Se detecta un "outlier" en "lenght" ["lenght (millimeters)"].

7) Varias columnas son asimétricas por la derecha. Con el logaritmo, se centrarían esas distribuciones y deaparecerían "outliers" ["weight (carat)", "lenght (millimeters)", "width (millimeters)" y "depth (millimeters)"].

8) Existe un valor atípico en "weight". Sin embargo, imputarlo al valor máximo de su "boxplot" haría que resurgiese después de un escalado estándar ["weight (carat)"].

9) Dos columnas tienen una alta correlación negativa que se quiere conservar, y muchos "outliers" no compartidos. Se presume que la posición de esos "outliers" es la que determina la correlación ["depth (percentage)" y "table (percentage)"].

10) "depth (millimeters)" tiene valores atípicos. Esta variable, junto con las demás relacionadas con el tamaño, están tan correlacionadas que casi podría considerarse que son la misma variable escalada ["depth (millimeters)"].

11) Algunas variables tienen valores mucho más altos comparadas con el resto.

----------------------------------

- Con esa información, se prueban una serie de posibles cambios a llevar a cabo durante la fase de "feature_engineering". Si se aplica el conjunto de estos cambios, el "dataframe" queda libre de nulos completamente:

1) Borrado de "outliers" extremadamente altos ["depth (percentage)", "table (percentage)", "width (millimeters)", "depth (millimeters)"].

2) Borrado de filas que tienen 0 en todas las variables de tamaño ["lenght (millimeters)", "width (millimeters)" y "depth (millimeters)"].

3) Borrado de los "outliers" compartidos moderadamente altos ["depth (percentage)" y "table (percentage)"].

4) Asignación del valor con 0 restante en "lenght" al "width" correspondiente ["lenght (millimeters)"].

5) Asignación del valor con 0 restante de "depth (millimeters)" a partir de una operación con el "lenght", el "width" y el "depth (percentage)" correspondientes ["depth (millimeters)"].

6) Asignación del "outlier" restante del "lenght" al "width" correspondiente ["lenght (millimeters)"].

7) Transformación con logaritmo ["weight (carat)", "lenght (millimeters)", "width (millimeters)" y "depth (millimeters)"].

8) Imputación al siguiente valor más alto ["weight (carat)"].

9) Imputación a los valores máximos y mínimos del "boxplot" ["depth (percentage)" y "table (percentage)"].

10) Neutralización de "outliers" con un modelo "ridge" ["depth (millimeters)"].

11) Escalado.

## Cambios no probados

- Se detectan una serie de características:

12) La variable "depth (percentage)" sale de dividir "depth (millimeters)" por la media de "lenght (millimeters)" y "width (millimeters)". Por tanto, si se hace la operación, el resultado debería coincidir con la columna de "depth (percentage)". Sin embargo, no es así. Habrá que probar durante el "feature engineering" si utilizar los valores calculados mejora la predicción.

13) Las columnas relacionadas con el tamaño tienen una correlación altísima. Quizá los resultados mejoren con borrarlas. Otras columnas tinen una correlación ínfima, cercana a 0, que podría considerarse irrelevante

14) Las filas con el valor máximo de "clarity quality" (7, variable discreta) no cambían en cuanto a las variables relacionadas con el tamaño si se comparan con las que tienen una "clarity quality" un punto por debajo (6). Quizá cabría imputar esos valores 7 de "clarity quality" al 6.


# Feature_engineering

- Cuando se aplican los cambios probados y no probados consecutivamente intercalando múltiples modelos para ver 1) qué modelos mejoran con qué cambios, y 2) qué modelo es el adecuado para la optimización

- Se lleva a cabo en el "notebook" homónimo

## Selección de cambios

- Se reúnen los cambios apuntados en el EDA en subrgupos:

a) Borrado: 1, 2 y 3 (borrado parcial: 2)

b) Asignación: 4, 5, 6 (asignación parcial: 4 y 5)

c) Logaritmo: 7

d) Valor más alto de "weight": 8

e) Imputaciones "boxplot": 9

f) Imputaciones "ridge": 10

g) Escalado "MinMax": 11

h) Escalad "Standard": 11

i) Sustitución: 12

j) Descarte correlación altísima: 13 -> ningún modelo mejora

j) Descarte correlación ínfima: 13

k) Imputaciones "clarity quality": 14

--------------------------------

- Cada uno de los modelos mejora con los siguientes cambios:


---------- LinearRegression ----------

· Borrado

· Asignación

· Logaritmo

· Imputaciones "boxplot"

· Imputaciones "ridge"

· Sustitución

· Imputaciones "clarity quality"

---------- Ridge ----------

· Escalado "MinMax"

· Borrado

· Asignación

· Logaritmo

· Imputaciones "boxplot"

· Imputaciones "ridge"

· Sustitución

· Imputaciones "clarity quality"

---------- KNeighborsRegressor ----------

· Escalado "Standard"

· Borrado parcial

· Asignación

· Valor más alto de "weight"

· Imputaciones "boxplot"

· Imputaciones "ridge"

· Sustitución

· Descarte correlación ínfima

· Imputaciones "clarity quality"

---------- SVR ----------

· Escalado "Standard"

· Borrado

· Asignación

· Logaritmo

· Valor más alto de "weight"

· Imputaciones "boxplot"

· Imputaciones "ridge"

· Sustitución

· Descarte correlación ínfima

---------- DecisionTree ----------

· Borrado parcial

· Asignación parcial

---------- RandomForest ----------

· Borrado parcial

· Asignación parcial

· Sustitución

---------- XGBRegressor ----------

· Borrado parcial

· Asignación parcial


## Selección de modelos

- Estos son los resultados, una vez aplicados los cambios y los hiperparámetros, si se compara la predicción con el "y_test":

a) "LinearRegression" mejora su "rmse" de 0.182664 a 0.143851 (-21.24%)

b) "Ridge" mejora su "rmse" de 0.182826 a 0.144234 (-21.11%)

c) k vecinos mejora su "rmse" de 0.181632 a 0.121120 (-33.32%)

d) "SVR" mejora su "rmse" de 0.189328 a 0.100319 (-47.01%)

e) "DecisionTree" mejora su "rmse" de 0.130480 a 0.109705 (-15.92%)

f) "RandomForest" mejora su "rmse" de 0.094246 a 0.091972 (-2.41%)

g) "XGBRegressor" mejora su "rmse" de 0.090762 a 0.086893 (-4.26%)

- Se elige trabajar en la siguiente fase con "XGB". Es el mejor con una "rmse" de 0.086893, y, como dispone de muchos más hiperparámetros que los probados hasta ahora, probablemente tenga un margen considerable de optimización


# Model_optimization

- Donde se experimenta de forma extensiva con "XGB" con tal de obtener los mejores resultados posibles. El modelo entrenado que sale es el que se usa para la competición

- Se lleva a cabo en el "notebook" homónimo

## Rondas de mejora

- Estos son los hiperparámetros que se añaden al modelo a cada ronda:

1) Se prueba sin modificación alguna para tener una base desde la que hacer comparaciones

2) Se prueba con los cambios de "feature engineering" (borrado parcial y asignación parcial) por la misma razón

3) Se obtiene un número de estimadores óptimo como punto de partida y se tunea el "learning rate" ("eta"). Una vez tuneado, se vuelve a buscar el número óptimo de estimadores

4) Se aplican "constaints" a la columna "weight", ya que solo crece con el precio

5) Se establecen un "subsample" y "colsample_bytree" de 0.8, que es el estándar recomendable, y se tunean "max_depth" y "min_child_weight"

6) Se tunea "gamma", parámetro de semiregularización

7) Se prueban dos opciones y se tunean: a) "sampling" combinado con "colsample_bytree", y b) "sampling_method='gradient_based" combinado con "colsample_bytree". Gana la opción "a"

8) Se tunean los hiperparámetros de regularización ("alpha", "lambda" y "max_delta_step"), que son los que ayudan a prevenir el "overfitting" y el "underfitting"

9) Se aumenta el número de árboles en paralelo, convirtiendo, a todos los efectos, el "xgboost" en un "random forest" con "booster"

10) Se reajusta el número de estimadores una vez más

## Resultado

- Se superan los puntos del ganador de la competición con una "rmse" de 0.08506 vs 0.08617

# Target_transformation

- Se descubre que la variable "target" del "dataframe" de competición se ha escalado, y no representa los precios reales de los diamantes

- El "dataset" con los precios reales se encuentra aquí: https://www.kaggle.com/datasets/swatikhedekar/price-prediction-of-diamond

- En el "notebook" homónimo se investiga si es cierto que los dos "datasets" son la misma cosa y qué tipo de escalado se ha usado con tal de poder revertirlo cuando sea necesario

- El escalado efectuado es el logaritmo